In [10]:
import dotenv

dotenv.load_dotenv()

True

In [11]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")


In [12]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
Analyze the every following code.
After anylyzing try to find some vulnerabilities in the code.
Here is a list of i wanna find: 
    - Reentrency Attack: Reentrancy is a vulnerability that allows an attacker to re-enter a function multiple times before the first function call is finished. so whenever the contract makes external call to other addresses, this is a possibility for reentrancy attack.This can lead to unexpected behavior, including reordering of transactions, and can be used to drain funds from a contract. 
    - Replay signatures attacks: The original account will sign a message then the delivery account will send the message to a smart contract, that way it is the delivery account that pays for the transaction fees and not the original account.
If you find anything, please provide a list of them in JSON format. Escape quotes in the JSON with a backslash. 
The JSON should contain the following fields:
- "line": The line number where the issue occurs.
- "token": The specific code token that is problematic.
- "problem": A short name for the issue.
- "severity": The severity of the issue (low, high).
- "explanation": A brief explanation of why this is a security risk.
- "migration": A brief explanation of how to fix this issue.
Code:
{code}
"""
)

In [13]:
import os

files = os.listdir('snippets')
code_snippets = []
for file in files:
    with open(f'snippets/{file}', 'r') as f:
        code_snippets.append(f.read())
print(code_snippets)

['// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBank{\n    mapping(address => uint) public credit;\n    \n    function deposit() public payable {\n        credit[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint amount) public{\n        if (credit[msg.sender]>= amount) {\n            require(msg.sender.call.value(amount)());\n            credit[msg.sender]-=amount;\n        }\n    }\n}', '// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBank is ReentrancyGuard {\n    mapping(address => uint) public credit;\n\n    function deposit() public payable {\n        credit[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint amount) public nonReentrant {\n        require(credit[msg.sender] >= amount, "Insufficient balance");\n        \n        credit[msg.sender] -= amount; // Effect: Update state before transferring funds\n        \n        (bool success,) = payable(msg.sender).call{value: amount}(""); // Interaction: T

In [14]:
from langchain.chains import LLMChain
chain = chain = prompt | model

In [15]:
code = code_snippets[2]

In [16]:
result = chain.invoke({"code": code})
result

AIMessage(content='Let\'s analyze the provided Solidity smart contract, `PiggyBridge`, for potential vulnerabilities, particularly focusing on reentrancy and replay attacks. Below is a breakdown of these issues and their respective details in the specified JSON format.\n\n### Analysis\n\n1. **Reentrancy Attack:** \n   - The `withdraw` function, despite using the `nonReentrant` modifier intended to prevent reentrancy, still performs a state change (decreasing the balance) after making an external call (`payable(msg.sender).transfer(amount)`). If any code that interacts with the `withdraw` function can manipulate the contract\'s flow, it could lead to unexpected behavior, especially in scenarios where the modifier might not be used correctly.\n\n2. **Replay Signature Attacks:**\n   - The contract does not incorporate a mechanism to prevent replay attacks across different contexts. If the format of the message or nonce handling is not unique for each transaction, a signed transaction coul

In [17]:
result_text = result.content.strip('`json')
print(code)
print(result_text)

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract PiggyBridge is ReentrancyGuard {
    mapping(address => uint256) public balances;
    mapping(bytes32 => bool) public executedTxs;

    function deposit() external payable {
        balances[msg.sender] += msg.value;
    }

    function withdraw(uint256 amount, uint256 nonce, bytes memory signature) external nonReentrant {
        require(balances[msg.sender] >= amount, "Insufficient balance");

        bytes32 messageHash = keccak256(abi.encodePacked(msg.sender, amount, nonce));
        require(!executedTxs[messageHash], "Already executed");

        address signer = recoverSigner(messageHash, signature);
        require(signer == msg.sender, "Invalid signature");

        executedTxs[messageHash] = true;
        balances[msg.sender] -= amount;
        payable(msg.sender).transfer(amount);
    }

    function recoverSigner(bytes32 hash, bytes memory signature) public pure returns (address) {
        bytes32 ethSignedHas

In [18]:
import json
(result_text[result_text.find("```json") + len("```json"):result_text.find("```", result_text.find("```json") + len("```json"))])
json.loads(result_text[result_text.find("```json") + len("```json"):result_text.find("```", result_text.find("```json") + len("```json"))])

[{'line': 24,
  'token': 'payable(msg.sender).transfer(amount)',
  'problem': 'Reentrancy Attack',
  'severity': 'high',
  'explanation': 'This external call can allow an attacker to re-enter the contract and cause unexpected state changes before the function call completes.',
  'migration': 'Ensure that all state changes occur before making external calls, or use the Checks-Effects-Interactions pattern.'},
 {'line': 18,
  'token': 'bytes32 messageHash = keccak256(abi.encodePacked(msg.sender, amount, nonce))',
  'problem': 'Replay Signature Attack',
  'severity': 'high',
  'explanation': 'The contract could allow the same signature to be reused across different calls to withdraw funds if nonces are not handled properly or globally unique.',
  'migration': 'Incorporate a unique nonce per transaction along with the message hash that is distinct per user to prevent reuse.'}]